### 🏎️ F1 퀄리파잉 심층 분석: 드라이버 & 시대적 진화

**분석 목표**
1. **팀메이트 배틀 (Teammate Battle):** 차량의 성능 차이를 배제하고, 동일한 차를 타는 드라이버 간의 퀄리파잉 랩타임 격차(Gap)를 분석하여 드라이버의 순수 퍼포먼스를 평가합니다.
2. **시대적 랩타임 진화 (Era Evolution):** 2018년부터 2025년까지 주요 서킷의 폴 포지션 기록 변화를 추적하여, 2022년 기술 규정 대격변(Ground Effect)이 차량 성능에 미친 영향을 정량화합니다.

In [15]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import plotly.graph_objects as go
from datetime import datetime

# === [설정] ===
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "f1_analytics_db"

client = MongoClient(MONGO_URI)
db = client[DB_NAME]

print(f"✅ MongoDB Connected: {DB_NAME}")

# 시간 문자열(1:23.456)을 초(Seconds) 단위로 변환하는 헬퍼 함수
def str_to_seconds(time_str):
    if pd.isna(time_str) or time_str == "" or "DNF" in str(time_str):
        return None
    try:
        # 분:초.밀리초 형식 (1:23.456)
        t = datetime.strptime(str(time_str), "%M:%S.%f")
        return t.minute * 60 + t.second + t.microsecond / 1e6
    except ValueError:
        try:
            # 초.밀리초 형식 (23.456) - Q1/Q2/Q3 탈락 시 이런 경우 있음
            return float(time_str)
        except:
            return None

✅ MongoDB Connected: f1_analytics_db


1. 분석 1 - 팀메이트 배틀 데이터 추출
    - 특정 팀/연도의 퀄리파잉(예선) 데이터를 조회하여 라운드 별 팀메이트 간의 기록 격차(초, %)를 계산합니다.

In [26]:
def str_to_seconds(time_val):
    """
    '1:23.456', '0 days 00:01:23.456', '83.456' 등 다양한 포맷을 초(float)로 변환
    """
    if pd.isna(time_val) or time_val == "" or "DNF" in str(time_val):
        return None

    # 1. 이미 숫자(float/int)인 경우 바로 반환
    if isinstance(time_val, (int, float)):
        return float(time_val)

    # 2. 문자열 처리
    time_str = str(time_val).strip()

    try:
        # Pandas의 강력한 파서 사용 ('0 days ...' 포맷도 자동 처리)
        dt = pd.to_timedelta(time_str)
        return dt.total_seconds()
    except:
        pass

    # 3. Pandas로 안 될 경우, 기존 방식 시도 (분:초.밀리초)
    try:
        t = datetime.strptime(time_str, "%M:%S.%f")
        return t.minute * 60 + t.second + t.microsecond / 1e6
    except:
        return None

def analyze_teammate_battle(year, team_keyword):
    print(f"⚔️ 팀메이트 배틀 분석 (v3): '{team_keyword}' ({year})...")

    # 팀 이름 찾기
    regex_query = {"$regex": team_keyword, "$options": "i"}
    matched_teams = db.results.distinct("Team", {"Year": year, "Team": regex_query})

    if not matched_teams:
        print(f"❌ 팀을 찾을 수 없습니다. (검색어: {team_keyword})")
        return pd.DataFrame()

    target_team = matched_teams[0]
    print(f"   👉 분석 대상 팀: {target_team}")

    cursor = db.results.find(
        {"Year": year, "Team": target_team, "SessionType": "Qualifying"},
        {"Round": 1, "Driver": 1, "Q1": 1, "Q2": 1, "Q3": 1, "Position": 1, "_id": 0}
    ).sort("Round", 1)

    df = pd.DataFrame(list(cursor))
    if df.empty:
        print("❌ 데이터가 비어 있습니다.")
        return pd.DataFrame()

    battle_data = []
    rounds = df['Round'].unique()

    for r in rounds:
        round_df = df[df['Round'] == r]
        if len(round_df) < 2: continue

        drivers_perf = []
        for _, row in round_df.iterrows():
            valid_time = None
            # Q3 -> Q2 -> Q1 순서로 유효 기록 탐색
            for q in ['Q3', 'Q2', 'Q1']:
                # [수정] v2 함수 사용
                t = str_to_seconds(row.get(q))
                if t:
                    valid_time = t
                    break

            if valid_time:
                drivers_perf.append({
                    "Driver": row['Driver'],
                    "Time": valid_time
                })

        # 두 드라이버 비교
        if len(drivers_perf) >= 2:
            d1 = drivers_perf[0]
            d2 = drivers_perf[1]

            if d1['Time'] < d2['Time']:
                winner, loser = d1, d2
            else:
                winner, loser = d2, d1

            gap = loser['Time'] - winner['Time']
            gap_pct = (gap / winner['Time']) * 100

            battle_data.append({
                "Round": r,
                "Winner": winner['Driver'],
                "Loser": loser['Driver'],
                "Gap_Sec": gap,
                "Gap_Pct": gap_pct
            })

    print(f"✅ 분석 완료: 총 {len(battle_data)}개 라운드 데이터 확보.")
    return pd.DataFrame(battle_data)

def visualize_teammate_battle(df, year, team):
    # [중요] 데이터가 비었으면 그래프를 그리지 않고 메시지 출력
    if df.empty:
        print("⚠️ 시각화할 데이터가 없습니다.")
        return

    # 기준 드라이버 설정 (가장 많이 이긴 사람을 위쪽으로)
    top_winner = df['Winner'].mode()[0]

    # 상대 드라이버 찾기
    others = df[df['Winner'] != top_winner]['Winner'].unique()
    if len(others) > 0:
        bottom_driver = others[0]
    else:
        # 전승한 경우, 패자 목록에서 찾기
        bottom_driver = df['Loser'].unique()[0]

    # 그래프용 데이터 변환
    plot_data = []
    colors = []

    for _, row in df.iterrows():
        # Top Driver가 이겼으면 양수(+), 졌으면 음수(-)
        if row['Winner'] == top_winner:
            val = row['Gap_Pct']
            colors.append('#00D2BE') # Cyan (승리)
        else:
            val = -row['Gap_Pct']
            colors.append('#DC0000') # Red (패배)
        plot_data.append(val)

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=df['Round'],
        y=plot_data,
        marker=dict(color=colors),
        text=[f"{abs(v):.2f}%" for v in plot_data], # 절댓값으로 표시
        textposition='auto',
        hovertemplate='<b>라운드 %{x}</b><br>격차: %{y:.3f}%<extra></extra>'
    ))

    fig.update_layout(
        title=f"⚔️ {team} 팀메이트 배틀 ({year}): {top_winner} (위) vs {bottom_driver} (아래)",
        xaxis_title="라운드 (Round)",
        yaxis_title="기록 격차 (%)",
        template="plotly_dark",
        yaxis=dict(
            title="팀메이트 간 격차 (%)",
            range=[-2.5, 2.5] # 격차 범위 고정 (보기 편하게)
        ),
        height=500,
        # 한글 폰트 설정
        font=dict(
            family="Malgun Gothic, AppleGothic, sans-serif",
            size=12,
            color="white"
        )
    )

    # 기준선 (0)
    fig.add_hline(y=0, line_width=2, line_color="white")

    fig.show()

# === 실행 및 시각화 ===
battle_df = analyze_teammate_battle(2023, "Red Bull")

if not battle_df.empty:
    visualize_teammate_battle(battle_df, 2023, "Red Bull Racing")
else:
    print("❌ 여전히 데이터가 없습니다. Step 1의 Raw Data를 확인해주세요.")

⚔️ 팀메이트 배틀 분석 (v3): 'Red Bull' (2023)...
   👉 분석 대상 팀: Red Bull Racing
✅ 분석 완료: 총 21개 라운드 데이터 확보.


**2023 Red Bull Racing - Verstappen vs Perez**
- 2023 컨스트럭터 챔피언(팀 1등)의 똑같은 차량(RB19)을 탑승한 두 드라이버 간의 순수 기량 차이를 보여준다.
- Y축을 시간(초)가 아닌 퍼센트(%) 격차로 변환하여, 서킷 길이에 따른 편향을 제거함.
- 분석
    - 압도적인 지배력
        - 시즌 중반부 이후부터 베르스타펜의 독주 체제가 데이터로 명확히 드러납니다.
    - Street King의 한계
        - 초반 라운드의 빨간색 막대가 페레즈가 시가지 서킷에서 강하다는 통설을 증명하지만, 시즌이 흐르고 유럽의 전통적인 서킷으로 넘어갈수록 격차가 급격히 벌어지는 일관성 부족 문제가 데이터로 확인됨

2. 시대적 랩타임 진화 데이터 추출
    - 지정된 주요 서킷들의 연도별 폴 포지션 기록을 추출하여 규정 변화에 따른 차량 성능 변화를 추적

In [18]:
def get_pole_evolution(circuits):
    print("⏳ Analyzing Pole Position Evolution (2018-2025)...")
    data = []

    for circuit_name in circuits:
        for year in range(2018, 2026):
            # 해당 연도, 해당 서킷의 퀄리파잉 찾기
            meta = db.races_meta.find_one({
                "Year": year, "SessionType": "Qualifying",
                "Circuit": {"$regex": circuit_name, "$options": "i"}
            })
            if not meta: continue

            # 폴 포지션 기록 찾기 (IsAccurate=True 인 것 중 1등)
            pole_lap = db.laps.find_one(
                {"Year": year, "Round": meta['Round'], "SessionType": "Qualifying", "IsAccurate": True},
                sort=[("LapTime_Sec", 1)]
            )

            if pole_lap:
                data.append({
                    "Circuit": circuit_name,
                    "Year": year,
                    "PoleTime": pole_lap['LapTime_Sec']
                })

    return pd.DataFrame(data)

In [19]:
def visualize_evolution(df):
    if df.empty:
        print("⚠️ 시각화할 데이터가 없습니다.")
        return

    fig = go.Figure()

    # 서킷 이름 한글 매핑 (필요시 추가)
    KOR_NAMES = {
        "Bahrain": "바레인", "Silverstone": "실버스톤", "Monza": "몬자",
        "Suzuka": "스즈카", "Barcelona": "바르셀로나"
    }

    for circuit in df['Circuit'].unique():
        subset = df[df['Circuit'] == circuit]
        korean_name = KOR_NAMES.get(circuit, circuit) # 매핑 없으면 영어 그대로

        fig.add_trace(go.Scatter(
            x=subset['Year'], y=subset['PoleTime'],
            mode='lines+markers',
            name=korean_name,
            line=dict(width=3)
        ))

    # 2022년 규정 대격변 표시
    fig.add_vline(
        x=2022,
        line_dash="dash",
        line_color="white",
        annotation_text="2022 기술 규정 대격변 (Ground Effect)",
        annotation_position="top left"
    )

    fig.update_layout(
        title="📉 F1 랩타임 진화 추이 (2018-2025): 폴 포지션 기록",
        xaxis_title="연도 (Year)",
        yaxis_title="랩타임 (초) [낮을수록 빠름]",
        template="plotly_dark",
        height=600,
        hovermode="x unified",
        # 한글 폰트 설정
        font=dict(
            family="Malgun Gothic, AppleGothic, sans-serif",
            size=12,
            color="white"
        )
    )
    # Y축 뒤집기 (시간은 작을수록 빠르므로)
    fig.update_yaxes(autorange="reversed")

    fig.show()

# === 실행 예시 ===
# 검색할 서킷 키워드 (부분 일치 검색)
target_circuits = ["Bahrain", "Silverstone", "Monza", "Suzuka", "Barcelona"]
evo_df = get_pole_evolution(target_circuits)
visualize_evolution(evo_df)

⏳ Analyzing Pole Position Evolution (2018-2025)...


**2018~2025 폴 포지션 기록 추이**
- 2018년부터 2025년까지의 폴 포지션 랩타임 변화를 시계열로 추적하여, FIA의 규정 변경(Regulation Change)이 차량 성능에 미친 영향력을 정량화
- Y축은 역순(위로 갈수록 기록이 낮음 = 빠름)으로 배치하여 직관적으로 '속도'의 변화를 표현함

* 2020년
    * F1 역사상 가장 빨랐던 차량들의 시대를 대변합니다. (Mercedes W11 등)
* 2022년
    * 점선으로 표시된 2022년 기점에서 그래프가 급격히 아래로 꺾입니다.
    * 원인 : 그라운드 이펙트(Ground Effect) 재도입, 18인치 휠 변경으로 인한 무게 증가, 그리고 포포이징(Porpoising)(차가 위아래로 심하게 출렁이는 현상) 이슈 해결을 위한 차고 조정 등이 복합적으로 작용하여 차량 성능이 물리적으로 봉인되었음을 보여줍니다.
* 2023~2024년
    * 2022년 이후 그래프가 다시 상승합니다.
    * 엔지니어들이 새로운 규정 내에서 다운포스를 되찾아가는 속도가 매우 빠름을 의미합니다. 불과 2년 만에 2022년 대비 랩타임을 1초 이상을 단축하였습니다.